# Introdução

Esse notebook serve de auxilio a atualização das tabelas vindo do [TSE](https://dadosabertos.tse.jus.br/dataset/?tags=Ano+2024)

Infelizmente no momento desse notebook o TSE não aceita acessos de Ips estrangeiros. Então, aqui está concetrada o tratamento dos dados e não o download dos arquivos brutos originais.

Temos algumas categorias diferente que são necessarias serem baixadas para a utilização desses codigos.

Fontes necessarias de exemplo:

- https://dadosabertos.tse.jus.br/dataset/candidatos-2024
- https://dadosabertos.tse.jus.br/dataset/resultados-2024

Apesar de existir duplicadas para serem tratadas e outros problemas menores com os dados originais. O tratamento é focado em deixa os dados no formato compativel com os dados já em produção e de anos mais antigos.

## Ordem de execução

### Celulas obrigatorias para execução do notebook

- Instalação
- Funções e utilitários
- Constantes

>  Não esqueça de atualizar caminhos de arquivos para os locais corretos da sua sessão ou local de execução e de atualizar o paramentro de `ano` das funçoes para o ano que gostaria de atualizar os dados

### Extrair

Conjuto de celulas não são obrigatorias. Serve mais como um auxilio caso necessite de ajuda para extrair arquivos de ZIPs. Formato o qual os arquivo vem do TSE

Là apenas tem exemplo de codigo e utilização dele.

### Tabelas

Cada tabela tem sua celular de funçoes e logo abaixo uma celula de execução de tratamento. Elas tevem ser executadas em sequencia sempre!

> Não esquecer de atualizar os caminhos de onde se encontra os arquivos CSVs originais do TSE. Existem comentarios avisando das alteraçoes necessarias


## Consideração final

Codigos não estão revisados e não são versoes finais. A ideia seria criar um `class` com a possibilidade de tratar todas as tabelas necessarias do TSE.
Porém, essa versão está aqui como auxilio e pode vim a ser atualizada.

Algumas outras tabelas são atualizadas a partir de codigos que estão no reposito de [pipelines](https://github.com/basedosdados/pipelines/blob/main/pipelines/utils/crawler_tse_eleicoes/utils.py) que podem ser rodados localmente.


# Instalação

In [ ]:
from IPython.display import clear_output
import os
import sys
import shutil

!pip install basedosdados==2.0.0b16
clear_output()


#  Funções e utilitários

In [9]:
from pathlib import Path, PurePath
import tempfile
import basedosdados as bd
import pandas as pd
import zipfile
import glob
import os
import unicodedata
from datetime import datetime
import shutil


def slugify(s: str) -> str:

    s = s.strip().lower()
    s = unicodedata.normalize("NFD", s)
    if "º" not in s:
      s = s.encode("ascii", "ignore")
      s = s.decode("utf-8")
    s = s.lower().strip()
    return s


def conv_data(date: str, birth: bool = False) -> str:
  try:
    data_datetime = datetime.strptime(date, "%d/%m/%Y")
    if birth:
      idade = datetime.now().year - data_datetime.year
      if not 18 <= idade < 120 :
        raise Exception("Idade Inválida")
    return data_datetime.strftime('%Y-%m-%d')
  except:
    return ""


def to_partitions(
    data: pd.DataFrame,
    partition_columns: list,
    savepath: str,
    file_type: str = "csv",
    name_file: str = "data"
):
    """Save data in to hive patitions schema, given a dataframe and a list of partition columns.
    Args:
        data (pandas.core.frame.DataFrame): Dataframe to be partitioned.
        partition_columns (list): List of columns to be used as partitions.
        savepath (str, pathlib.PosixPath): folder path to save the partitions.
        file_type (str): default to csv. Accepts parquet.
    Exemple:
        data = {
            "ano": [2020, 2021, 2020, 2021, 2020, 2021, 2021,2025],
            "mes": [1, 2, 3, 4, 5, 6, 6,9],
            "sigla_uf": ["SP", "SP", "RJ", "RJ", "PR", "PR", "PR","PR"],
            "dado": ["a", "b", "c", "d", "e", "f", "g",'h'],
        }
        to_partitions(
            data=pd.DataFrame(data),
            partition_columns=['ano','mes','sigla_uf'],
            savepath='partitions/',
        )
    """

    if isinstance(data, (pd.core.frame.DataFrame)):
        savepath = Path(savepath)
        # create unique combinations between partition columns
        unique_combinations = (
            data[partition_columns]
            # .astype(str)
            .drop_duplicates(subset=partition_columns).to_dict(orient="records")
        )

        for filter_combination in unique_combinations:
            patitions_values = [
                f"{partition}={value}"
                for partition, value in filter_combination.items()
            ]

            # get filtered data
            df_filter = data.loc[
                data[filter_combination.keys()]
                .isin(filter_combination.values())
                .all(axis=1),
                :,
            ]
            df_filter = df_filter.drop(columns=partition_columns)

            # create folder tree
            filter_save_path = Path(savepath / "/".join(patitions_values))
            filter_save_path.mkdir(parents=True, exist_ok=True)

            if file_type == "csv":
                # append data to csv
                file_filter_save_path = Path(filter_save_path) / f"{name_file}.csv"
                df_filter.to_csv(
                    file_filter_save_path,
                    sep=",",
                    encoding="utf-8",
                    na_rep="",
                    index=False,
                    mode="a",
                    header=not file_filter_save_path.exists(),
                )
            elif file_type == "parquet":
                # append data to parquet
                file_filter_save_path = Path(filter_save_path) / f"{name_file}.parquet"
                df_filter.to_parquet(
                    file_filter_save_path, index=False, compression="gzip"
                )
    else:
        raise BaseException("Data need to be a pandas DataFrame")


def send_folder_drive(path_drive: str, path_input: str) -> None:

  if not len(os.listdir(path_input)) > 0:
    raise Exception("Essa Pasta vazia")
    return None

  shutil.make_archive(path_drive,
                      'zip',
                      path_input)

# Constantes

In [21]:

QUERY_MUNIPIPIOS = "select id_municipio, id_municipio_tse from `basedosdados.br_bd_diretorios_brasil.municipio`"

municipios = bd.read_sql(
  QUERY_MUNIPIPIOS,
  billing_project_id=None # Colocar o nome do seu projeto
)

path_candidatos = None # Colocar caminho para arquivo com todos os candidatos. Exemplo "votacao_candidato_munzona_2024_BRASIL.csv"
candidatos = pd.read_csv(path_candidatos,  sep=";", encoding="ISO-8859-1", dtype=str, usecols=["NR_PARTIDO", "SG_PARTIDO", "SQ_CANDIDATO"])

# Tabelas

## Extrair

In [ ]:
def extrair_drive(zip_name: str) -> None:
  path_base_input_zip = None # atualizar caminho base onde se encontra os ZIPs do tse

  path_zip_file = path_base_input_zip + zip_name

  path_base_input = Path("input")
  path_base_input.mkdir(parents=True, exist_ok=True)

  path_base_input = path_base_input / zip_name.split(".")[0]

  with zipfile.ZipFile(path_zip_file) as z:
      z.extractall(path_base_input)

In [ ]:
zip_names = ["votacao_partido_munzona_2024.zip",
             "votacao_candidato_munzona_2024.zip",
             "detalhe_votacao_secao_2024.zip",
             "detalhe_votacao_munzona_2024.zip",
             "resultados_atualizada_2024.zip",
             "resultados_atualizado_secao_2024.zip"]

for zip_name in zip_names:
  extrair_drive(zip_name)

In [23]:
extrair_drive("secao_input.zip")

In [ ]:
extrair_drive("resultados_atualizado_secao_2024.zip")

## resultados_candidato

In [11]:
ORDER = { 'turno': 'NR_TURNO',
          'id_eleicao': 'CD_ELEICAO',
          'tipo_eleicao': 'NM_TIPO_ELEICAO',
          'data_eleicao': 'DT_ELEICAO',
          'sigla_uf': 'SG_UF',
          'id_municipio': 'id_municipio',
          'id_municipio_tse': 'CD_MUNICIPIO',
          'cargo': 'DS_CARGO',
          'numero_partido': 'NR_PARTIDO',
          'sigla_partido': 'SG_PARTIDO',
          'numero_candidato': 'NR_CANDIDATO',
          'sequencial_candidato': 'SQ_CANDIDATO',
          'id_candidato_bd': 'id_candidato_bd',
          'nome_candidato': 'NM_CANDIDATO',
          'resultado': "DS_SIT_TOT_TURNO",
          'votos': "votos"}


def get_count_votos(temp_df: pd.DataFrame, turno: str|int) -> pd.DataFrame:

  clone_temp = temp_df[temp_df.NR_TURNO == turno].copy()
  clone_temp["QT_VOTOS_NOMINAIS"] = clone_temp["QT_VOTOS_NOMINAIS"].astype(int)
  contagem = pd.DataFrame(
      clone_temp.groupby("SQ_CANDIDATO")['QT_VOTOS_NOMINAIS'].sum()).reset_index()

  contagem.columns =  ["sequencial_candidato_contagem", "votos"]

  return contagem

def make_turno_table(temp_df: pd.DataFrame, turno: str|int):

  df_principal = temp_df[temp_df.NR_TURNO == turno].copy()
  df_principal['id_candidato_bd'] = ""
  contagem = get_count_votos(temp_df, turno)


  temp_merge_left = pd.merge(df_principal, contagem, left_on="SQ_CANDIDATO", right_on='sequencial_candidato_contagem', how='left')
  temp_merge_left = pd.merge(temp_merge_left, municipios, left_on="CD_MUNICIPIO", right_on="id_municipio_tse", how="left")

  temp_merge_left = temp_merge_left.loc[:, ORDER.values()]
  temp_merge_left.columns = ORDER.keys()
  temp_merge_left.drop_duplicates(inplace=True)

  slug_columns_format = ["tipo_eleicao", "cargo", "resultado"]

  temp_merge_left[slug_columns_format] = temp_merge_left[slug_columns_format].applymap(slugify)

  temp_merge_left["nome_candidato"] = temp_merge_left["nome_candidato"].str.title()

  temp_merge_left["data_eleicao"] = temp_merge_left["data_eleicao"].apply(conv_data)


  return temp_merge_left

def processing_resultados_candidato(path_csvs: list, ano: int|str = 2024) -> None:

  temp_df =  pd.concat([pd.read_csv(path_csv,  sep=";", encoding="ISO-8859-1", dtype=str) for path_csv in path_csvs], ignore_index=False)

  path_output = Path("output")
  path_partition = path_output / f"ano={ano}"
  path_partition.mkdir(parents=True, exist_ok=True)

  tabela_resultados = pd.concat([make_turno_table(temp_df, turno) for turno in temp_df.NR_TURNO.unique()], ignore_index=True)
  output_name_file = path_partition / "resultados_candidato.csv"

  tabela_resultados.to_csv(output_name_file, index=False)


In [ ]:
csvs_match = None # Atualizar o caminho para onde está seus CSVs. Exemplo: "/content/input/resultados_atualizada_2024/*.csv"
csvs_files = [csv for csv in glob.glob(csvs_match) if "BRASIL" not in csv]
processing_resultados_candidato(csvs_files)

## resultados_candidato_municipio
Precisa de ajuste caso a eleição tem presidente

In [13]:

ORDER_MUNICIPIO = {'turno': 'NR_TURNO',
          'id_eleicao': 'CD_ELEICAO',
          'tipo_eleicao': 'NM_TIPO_ELEICAO',
          'data_eleicao': 'DT_ELEICAO',
          'sigla_uf': 'SG_UF',
          'id_municipio': 'id_municipio',
          'id_municipio_tse': 'CD_MUNICIPIO',
          'cargo': 'DS_CARGO',
          'numero_partido': 'NR_PARTIDO',
          'sigla_partido': 'SG_PARTIDO',
          'numero_candidato': 'NR_CANDIDATO',
          'sequencial_candidato': 'SQ_CANDIDATO',
          'id_candidato_bd': 'id_candidato_bd',
          'resultado': "DS_SIT_TOT_TURNO",
          'votos': "votos"}



def get_count_votos_municipio(temp_df: pd.DataFrame, turno: str|int) -> pd.DataFrame:

  clone_temp = temp_df[temp_df.NR_TURNO == turno].copy()
  clone_temp["QT_VOTOS_NOMINAIS"] = clone_temp["QT_VOTOS_NOMINAIS"].astype(int)
  contagem = pd.DataFrame(
      clone_temp.groupby("SQ_CANDIDATO")['QT_VOTOS_NOMINAIS'].sum()).reset_index()

  contagem.columns =  ["sequencial_candidato_contagem", "votos"]

  return contagem

def make_turno_table_municipio(temp_df: pd.DataFrame, turno: str|int, ano: int|str = 2024):

  df_principal = temp_df[temp_df.NR_TURNO == turno].copy()
  df_principal['id_candidato_bd'] = ""
  contagem = get_count_votos_municipio(temp_df, turno)


  temp_merge_left = pd.merge(df_principal, contagem, left_on="SQ_CANDIDATO", right_on='sequencial_candidato_contagem', how='left')
  temp_merge_left = pd.merge(temp_merge_left, municipios, left_on="CD_MUNICIPIO", right_on="id_municipio_tse", how="left")

  temp_merge_left = temp_merge_left.loc[:, ORDER_MUNICIPIO.values()]
  temp_merge_left.columns = ORDER_MUNICIPIO.keys()
  temp_merge_left.drop_duplicates(inplace=True)

  slug_columns_format = ["tipo_eleicao", "cargo", "resultado"]

  temp_merge_left[slug_columns_format] = temp_merge_left[slug_columns_format].applymap(slugify)

  temp_merge_left["data_eleicao"] = temp_merge_left["data_eleicao"].apply(conv_data)

  temp_merge_left["ano"] = str(ano)

  return temp_merge_left


def processing_resultados_candidato_municipio(path_csv: str, ano: int|str = 2024) -> None:

  temp_df = pd.read_csv(path_csv,  sep=";", encoding="ISO-8859-1", dtype=str)

  path_output = Path("output")
  path_output.mkdir(parents=True, exist_ok=True)

  tabela_resultados_candidato_municipio = pd.concat([make_turno_table_municipio(temp_df, turno, ano) for turno in temp_df.NR_TURNO.unique()])

  to_partitions(tabela_resultados_candidato_municipio,
                savepath="/content/output", # Atualizar caminho para sua saida de dados
                partition_columns=["ano", "sigla_uf"],
              name_file="resultados_candidato_municipio")

In [14]:
csvs_match = None # Atualizar o caminho para onde está seus CSVs. Exemplo: "/content/input/resultados_atualizada_2024/*.csv"
csvs_files = [csv for csv in glob.glob(csvs_match) if "BRASIL" not in csv]

for csv in csvs_files:
  processing_resultados_candidato_municipio(csv)

## resultados_candidato_municipio_zona

In [16]:

ORDER_MUNICIPIO_ZONA = {'turno': 'NR_TURNO',
          'id_eleicao': 'CD_ELEICAO',
          'tipo_eleicao': 'NM_TIPO_ELEICAO',
          'data_eleicao': 'DT_ELEICAO',
          'sigla_uf': 'SG_UF',
          'id_municipio': 'id_municipio',
          'id_municipio_tse': 'CD_MUNICIPIO',
          'cargo': 'DS_CARGO',
          'zona': 'NR_ZONA',
          'numero_partido': 'NR_PARTIDO',
          'sigla_partido': 'SG_PARTIDO',
          'numero_candidato': 'NR_CANDIDATO',
          'sequencial_candidato': 'SQ_CANDIDATO',
          'id_candidato_bd': 'id_candidato_bd',
          'resultado': "DS_SIT_TOT_TURNO",
          'votos': "QT_VOTOS_NOMINAIS"}


def make_turno_table_municipio_zona(temp_df: pd.DataFrame, turno: str|int, ano: int|str = 2024):

  df_principal = temp_df.copy()
  df_principal['id_candidato_bd'] = ""

  temp_merge_left = pd.merge(df_principal, municipios, left_on="CD_MUNICIPIO", right_on="id_municipio_tse", how="left")

  temp_merge_left = temp_merge_left.loc[:, ORDER_MUNICIPIO_ZONA.values()]
  temp_merge_left.columns = ORDER_MUNICIPIO_ZONA.keys()
  temp_merge_left.drop_duplicates(inplace=True)

  slug_columns_format = ["tipo_eleicao", "cargo", "resultado"]

  temp_merge_left[slug_columns_format] = temp_merge_left[slug_columns_format].applymap(slugify)

  temp_merge_left["data_eleicao"] = temp_merge_left["data_eleicao"].apply(conv_data)

  temp_merge_left["ano"] = str(ano)

  return temp_merge_left


def processing_resultados_candidato_municipio_zona(path_csv: str, ano: int|str = 2024) -> None:

  temp_df = pd.read_csv(path_csv,  sep=";", encoding="ISO-8859-1", dtype=str)

  path_output = Path("output")
  path_output.mkdir(parents=True, exist_ok=True)

  tabela_resultados_candidato_municipio_zona = pd.concat(
      [make_turno_table_municipio_zona(temp_df, turno, ano)
      for turno in temp_df.NR_TURNO.unique()])

  combinacao = ['ano', 'turno', 'id_eleicao', 'sigla_uf', 'id_municipio_tse', 'cargo', 'zona', 'sequencial_candidato', 'numero_candidato']

  tabela_resultados_candidato_municipio_zona.drop_duplicates(subset=combinacao, inplace=True)

  to_partitions(tabela_resultados_candidato_municipio_zona,
                savepath="/content/output",
                partition_columns=["ano", "sigla_uf"],
                name_file="resultados_candidato_municipio_zona")

In [17]:
csvs_match = None # Atualizar o caminho para onde está seus CSVs. Exemplo: "/content/input/resultados_atualizada_2024/*.csv"
csvs_files = [csv for csv in glob.glob(csvs_match) if "BRASIL" not in csv]

for csv in csvs_files:
  processing_resultados_candidato_municipio_zona(csv)

## resultados_candidato_secao

In [18]:

ORDER__CANDIDATO_SECAO = {'turno': 'NR_TURNO',
          'id_eleicao': 'CD_ELEICAO',
          'tipo_eleicao': 'NM_TIPO_ELEICAO',
          'data_eleicao': 'DT_ELEICAO',
          'id_municipio': 'id_municipio',
          'id_municipio_tse': 'CD_MUNICIPIO',
          'zona': 'NR_ZONA',
          'secao': 'NR_SECAO',
          'cargo': 'DS_CARGO',
          'numero_partido': 'NR_PARTIDO',
          'sigla_partido': 'SG_PARTIDO',
          'numero_candidato': 'NR_VOTAVEL',
          'sequencial_candidato': "SQ_CANDIDATO",
          'id_candidato_bd': 'id_candidato_bd',
          'votos': "QT_VOTOS",
          'sigla_uf': 'SG_UF',
}


def make_turno_table_municipio_secao(temp_df: pd.DataFrame, turno: str|int, ano: int|str = 2024):

  df_principal = temp_df[temp_df.NR_TURNO == turno].copy()
  df_principal['id_candidato_bd'] = ""

  temp_merge_left = pd.merge(df_principal, candidatos, left_on="SQ_CANDIDATO", right_on="SQ_CANDIDATO", how="left")
  temp_merge_left = pd.merge(temp_merge_left, municipios, left_on="CD_MUNICIPIO", right_on="id_municipio_tse", how="left")

  temp_merge_left = temp_merge_left.loc[:, ORDER__CANDIDATO_SECAO.values()]
  temp_merge_left.columns = ORDER__CANDIDATO_SECAO.keys()
  temp_merge_left.drop_duplicates(inplace=True)

  slug_columns_format = ["tipo_eleicao", "cargo"]

  temp_merge_left[slug_columns_format] = temp_merge_left[slug_columns_format].applymap(slugify)

  temp_merge_left["data_eleicao"] = temp_merge_left["data_eleicao"].apply(conv_data)

  temp_merge_left["ano"] = str(ano)

  temp_merge_left = temp_merge_left[~temp_merge_left.sequencial_candidato.isin(["-3", "-1"])]

  return temp_merge_left


def processing_chunk_partitions(path_csv: str) -> None:
  temp_data = pd.read_csv(path_csv,  sep=";", encoding="ISO-8859-1", dtype=str, chunksize=250000)

  for data in temp_data:

    ano = 2024

    path_output = Path("output")
    path_output.mkdir(parents=True, exist_ok=True)

    tabela_resultados_municipio_secao = pd.concat([make_turno_table_municipio_secao(data, turno, ano) for turno in data.NR_TURNO.unique()])

    to_partitions(tabela_resultados_municipio_secao,
                  savepath="/content/output",
                  partition_columns=["ano", "sigla_uf"],
                  name_file="resultados_candidato_secao")


In [ ]:
csv_file_match = None # Atualizar o caminho para onde está seus CSVs. Exemplo: "/content/input/secao_input/secao_input/*.csv"
csv_files = glob.glob(csv_file_match)

for path_file in csv_files:
  processing_chunk_partitions(path_file)
  print(f"Finalizado com sucesso {path_file}")

## resultados_partido_municipio

In [28]:
ORDER__PARTIDO_MUNICIPIO = {'turno': 'NR_TURNO',
          'id_eleicao': 'CD_ELEICAO',
          'tipo_eleicao': 'NM_TIPO_ELEICAO',
          'data_eleicao': 'DT_ELEICAO',
          'id_municipio': 'id_municipio',
          'id_municipio_tse': 'CD_MUNICIPIO',
          'cargo': 'DS_CARGO',
          'numero_partido': 'NR_PARTIDO',
          'sigla_partido': 'SG_PARTIDO',
          'votos_nominais': "votos_nominais",
          'votos_legenda': "votos_legenda",
          'sigla_uf': 'SG_UF',
}

def get_count_votos_partido_municipio(temp_df: pd.DataFrame, turno: str|int) -> pd.DataFrame:

  clone_temp = temp_df[temp_df.NR_TURNO == turno].copy()
  clone_temp["QT_VOTOS_NOMINAIS_VALIDOS"] = clone_temp["QT_VOTOS_NOMINAIS_VALIDOS"].astype(int)
  clone_temp["QT_VOTOS_LEGENDA_VALIDOS"] = clone_temp["QT_VOTOS_LEGENDA_VALIDOS"].astype(int)

  contagem = pd.DataFrame(
      clone_temp.groupby(["NR_PARTIDO", "CD_CARGO", "CD_MUNICIPIO"])[['QT_VOTOS_NOMINAIS_VALIDOS', 'QT_VOTOS_LEGENDA_VALIDOS']].sum()).reset_index()

  contagem.columns =  ["numero_partido", "cargo", "id_municipio_tse", "votos_nominais", "votos_legenda"]

  return contagem

def make_turno_table_partido_municipio(temp_df: pd.DataFrame, turno: str|int, ano: int|str = 2024):

  df_principal = temp_df.copy()

  contagem = get_count_votos_partido_municipio(temp_df, turno)

  temp_merge_left = pd.merge(
      df_principal, contagem,
      left_on=["NR_PARTIDO", "CD_CARGO", "CD_MUNICIPIO"],
      right_on=["numero_partido", "cargo", "id_municipio_tse"], how='left')

  temp_merge_left = pd.merge(temp_merge_left, municipios, left_on="CD_MUNICIPIO", right_on="id_municipio_tse", how="left")

  temp_merge_left = temp_merge_left.loc[:, ORDER__PARTIDO_MUNICIPIO.values()]
  temp_merge_left.columns = ORDER__PARTIDO_MUNICIPIO.keys()

  temp_merge_left.drop_duplicates(inplace=True)

  slug_columns_format = ["tipo_eleicao", "cargo"]

  temp_merge_left[slug_columns_format] = temp_merge_left[slug_columns_format].applymap(slugify)

  temp_merge_left["data_eleicao"] = temp_merge_left["data_eleicao"].apply(conv_data)

  temp_merge_left["ano"] = str(ano)

  return temp_merge_left



In [29]:
csvs_match_secao = None # Atualizar o caminho para onde está seus CSVs. Exemplo: "/content/input/resultados_atualizado_secao_2024/*.csv"
csvs_files_secao = [csv for csv in glob.glob(csvs_match_secao) if "BRASIL" not in csv]

ano = 2024

temp_df =  pd.concat([pd.read_csv(path_csv,  sep=";", encoding="ISO-8859-1", dtype=str) for path_csv in csvs_files_secao], ignore_index=False)

path_output = Path("output")
path_output.mkdir(parents=True, exist_ok=True)

tabela_resultados_partido_municipio = pd.concat(
    [make_turno_table_partido_municipio(temp_df, turno, ano)
    for turno in temp_df.NR_TURNO.unique()])

combinacao = ['ano', 'turno', 'id_eleicao', 'id_municipio_tse', 'cargo', 'numero_partido']

tabela_resultados_partido_municipio.drop_duplicates(subset=combinacao, inplace=True)


to_partitions(tabela_resultados_partido_municipio,
              savepath="/content/output",
              partition_columns=["ano", "sigla_uf"],
              name_file="resultados_partido_municipio")


## resultados_partido_municipio_zona

In [31]:
ORDER__PARTIDO_MUNICIPIO_ZONA = {'turno': 'NR_TURNO',
          'id_eleicao': 'CD_ELEICAO',
          'tipo_eleicao': 'NM_TIPO_ELEICAO',
          'data_eleicao': 'DT_ELEICAO',
          'id_municipio': 'id_municipio',
          'id_municipio_tse': 'CD_MUNICIPIO',
          'zona': 'NR_ZONA',
          'cargo': 'DS_CARGO',
          'numero_partido': 'NR_PARTIDO',
          'sigla_partido': 'SG_PARTIDO',
          'votos_nominais': "QT_VOTOS_NOMINAIS_VALIDOS",
          'votos_legenda': "QT_VOTOS_LEGENDA_VALIDOS",
          'sigla_uf': 'SG_UF',
}


def make_turno_table_partido_municipio_zona(temp_df: pd.DataFrame, turno: str|int, ano: int|str = 2024):

  df_principal = temp_df.copy()

  temp_merge_left = pd.merge(df_principal, municipios, left_on="CD_MUNICIPIO", right_on="id_municipio_tse", how="left")

  temp_merge_left = temp_merge_left.loc[:, ORDER__PARTIDO_MUNICIPIO_ZONA.values()]
  temp_merge_left.columns = ORDER__PARTIDO_MUNICIPIO_ZONA.keys()
  temp_merge_left.drop_duplicates(inplace=True)

  slug_columns_format = ["tipo_eleicao", "cargo"]

  temp_merge_left[slug_columns_format] = temp_merge_left[slug_columns_format].applymap(slugify)

  temp_merge_left["data_eleicao"] = temp_merge_left["data_eleicao"].apply(conv_data)

  temp_merge_left["ano"] = str(ano)

  return temp_merge_left



In [32]:
path_csv = None # Atualizar o caminho para onde está seus CSVs. Exemplo: "/content/input/resultados_atualizado_secao_2024/votacao_partido_munzona_2024_BRASIL.csv"

temp_df = pd.read_csv(path_csv,  sep=";", encoding="ISO-8859-1", dtype=str)

ano = 2024

path_output = Path("output")
path_output.mkdir(parents=True, exist_ok=True)

tabela_resultados_partido_municipio_zona = pd.concat(
    [make_turno_table_partido_municipio_zona(temp_df, turno, ano)
    for turno in temp_df.NR_TURNO.unique()])


combinacao = ['ano', 'turno', 'id_eleicao', 'id_municipio_tse', 'zona', 'cargo', 'numero_partido']

tabela_resultados_partido_municipio_zona.drop_duplicates(subset=combinacao, inplace=True)

to_partitions(tabela_resultados_partido_municipio_zona,
              savepath="/content/output",
              partition_columns=["ano", "sigla_uf"],
              name_file="resultados_partido_municipio_zona")


## resultados_partido_secao

In [36]:
ORDER__PARTIDO_SECAO = {'turno': 'NR_TURNO',
          'id_eleicao': 'CD_ELEICAO',
          'tipo_eleicao': 'NM_TIPO_ELEICAO',
          'data_eleicao': 'DT_ELEICAO',
          'id_municipio': 'id_municipio',
          'id_municipio_tse': 'CD_MUNICIPIO',
          'zona': 'NR_ZONA',
          'secao': 'NR_SECAO',
          'cargo': 'DS_CARGO',
          'numero_partido': 'NR_PARTIDO',
          'sigla_partido': 'SG_PARTIDO',
          'numero_candidato': 'NR_VOTAVEL',
          'sequencial_candidato': "SQ_CANDIDATO",
          'id_candidato_bd': 'id_candidato_bd',
          'votos': "QT_VOTOS",
          'sigla_uf': 'SG_UF',
}

ORDER__PARTIDO_SECAO_FINAL = ['turno',
          'id_eleicao',
          'tipo_eleicao',
          'data_eleicao',
          'id_municipio',
          'id_municipio_tse',
          'zona',
          'secao',
          'cargo',
          'numero_partido',
          'sigla_partido',
          'votos_nominais',
          'votos_legenda',
          'sigla_uf',
          'ano']


def make_turno_table_partido_secao(temp_df: pd.DataFrame, turno: str|int, ano: int|str = 2024):

  df_principal = temp_df[temp_df.NR_TURNO == turno].copy()
  df_principal['id_candidato_bd'] = ""

  temp_merge_left = pd.merge(df_principal, candidatos, left_on="SQ_CANDIDATO", right_on="SQ_CANDIDATO", how="left")
  temp_merge_left = pd.merge(temp_merge_left, municipios, left_on="CD_MUNICIPIO", right_on="id_municipio_tse", how="left")

  temp_merge_left = temp_merge_left.loc[:, ORDER__PARTIDO_SECAO.values()]
  temp_merge_left.columns = ORDER__PARTIDO_SECAO.keys()
  temp_merge_left.drop_duplicates(inplace=True)

  slug_columns_format = ["tipo_eleicao", "cargo"]

  temp_merge_left[slug_columns_format] = temp_merge_left[slug_columns_format].applymap(slugify)

  temp_merge_left["data_eleicao"] = temp_merge_left["data_eleicao"].apply(conv_data)

  temp_merge_left["ano"] = str(ano)

  # Contagem
  clone_temp = temp_merge_left
  clone_temp["votos"] = clone_temp["votos"].astype(int)

  contagem = pd.DataFrame(
      clone_temp.groupby(['turno', 'id_eleicao', 'tipo_eleicao', 'data_eleicao',
                          'id_municipio', 'id_municipio_tse', 'zona', 'secao', 'cargo',
                          'numero_partido', 'sigla_partido',  "ano", "sigla_uf"])[['votos']].sum()).reset_index()

  contagem.columns =  ['turno', 'id_eleicao', 'tipo_eleicao', 'data_eleicao', 'id_municipio',
                      'id_municipio_tse', 'zona', 'secao', 'cargo', 'numero_partido',
                      'sigla_partido',  "ano", "sigla_uf", "votos_nominais"]

  # Contagem Legenda
  clone_temp_legenda = temp_merge_left[temp_merge_left.sequencial_candidato== "-3"]
  clone_temp_legenda["votos"] = clone_temp_legenda["votos"].astype(int)

  contagem_legenda = pd.DataFrame(
      clone_temp_legenda.groupby(['id_municipio_tse', 'zona', 'secao', 'numero_candidato', 'cargo'])[['votos']].sum()).reset_index()

  contagem_legenda.columns =  ['id_municipio_tse', 'zona', 'secao', 'numero_partido', 'cargo', "votos_legenda"]

  contagem_temp_merge_left = pd.merge(contagem, contagem_legenda, left_on= ['id_municipio_tse', 'zona', 'secao', 'numero_partido', 'cargo'],
                                    right_on=['id_municipio_tse', 'zona', 'secao', 'numero_partido', 'cargo'], how="left")

  # Tratamento

  contagem_temp_merge_left['votos_legenda'] = contagem_temp_merge_left['votos_legenda'].fillna(0).astype(int)

  contagem_temp_merge_left = contagem_temp_merge_left.loc[:, ORDER__PARTIDO_SECAO_FINAL]
  contagem_temp_merge_left.columns = ORDER__PARTIDO_SECAO_FINAL

  contagem_temp_merge_left.drop_duplicates(inplace=True)

  return contagem_temp_merge_left


def processing_chunk_partitions_partido_secao(path_csv: str) -> None:

  temp_data = pd.read_csv(path_csv,  sep=";", encoding="ISO-8859-1", dtype=str, chunksize=250000)

  for data in temp_data:

    ano = 2024

    path_output = Path("output")
    path_output.mkdir(parents=True, exist_ok=True)

    tabela_resultados_municipio_secao = pd.concat([make_turno_table_partido_secao(data, turno, ano) for turno in data.NR_TURNO.unique()])

    to_partitions(tabela_resultados_municipio_secao,
                  savepath="/content/output",
                  partition_columns=["ano", "sigla_uf"],
                  name_file="resultados_partido_secao")


In [ ]:
csv_file_match = None # Atualizar o caminho para onde está seus CSVs. Exemplo: "/content/input/secao_input/secao_input/*.csv"
csv_files = glob.glob(csv_file_match)

for path_file in csv_files:
  processing_chunk_partitions_partido_secao(path_file)
  print(f"Finalizado com sucesso {path_file}")

In [ ]:
# Esses dados tem a ter muitas duplicadas e foi colocada uma nova camada de remoção deles
csv_file_match = None # Atualizar o caminho para onde está seus CSVs. Exemplo: "/content/output/ano=2024/**/*.csv"
csv_files = glob.glob(csv_file_match)

for csv in csv_files:
  drop_dupli = pd.read_csv(csv, dtype=str)
  colunas_drop = ["turno", 'id_eleicao', 'id_municipio_tse', 'zona', 'secao', 'cargo', 'numero_partido']

  drop_dupli.drop_duplicates(subset=colunas_drop, inplace=True)

  drop_dupli.to_csv(csv, index=False)
  print(f"Tirei duplicados do {csv}")